Tarea: Proyecto I. Parte 3 (Core)

Nombre: Luciano Benjamín Recalde Carballo

#### Carga de Datos


In [2]:
import pandas as pd
df = pd.read_csv('data/retail_sales_dataset.csv')
# Genera estadísticas descriptivas del DataFrame utilizando el método describe().
df.describe()

,Transaction ID,Age,Quantity,Price per Unit,Total Amount
count,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,500.500000,41.39200,2.514000,179.890000,456.000000
std,288.819436,13.68143,1.132734,189.681356,559.997632
min,1.000000,18.00000,1.000000,25.000000,25.000000
25%,250.750000,29.00000,1.000000,30.000000,60.000000
50%,500.500000,42.00000,3.000000,50.000000,135.000000
75%,750.250000,53.00000,4.000000,300.000000,900.000000
max,1000.000000,64.00000,4.000000,500.000000,2000.000000


Buscamos valores nulos

In [3]:
def get_nulll_data(df):
    qsna = df.shape[0] - df.isnull().sum(axis=0)
    qna = df.isnull().sum(axis=0)
    ppna = round(100 * (df.isnull().sum(axis=0) / df.shape[0]), 2)
    aux = {'datos sin NAs en q': qsna, 'Na en q': qna, 'Na en %': ppna}
    na = pd.DataFrame(data=aux)

    return na.sort_values(by='Na en %', ascending=False)
get_nulll_data(df)

,datos sin NAs en q,Na en q,Na en %
Transaction ID,1000,0,0.0
Date,1000,0,0.0
Customer ID,1000,0,0.0
Gender,1000,0,0.0
Age,1000,0,0.0
Product Category,1000,0,0.0
Quantity,1000,0,0.0
Price per Unit,1000,0,0.0
Total Amount,1000,0,0.0


No existen datos nulos en el dataset.

#### Transformación y Agregacion


Se agrega una columna para tener el valor total calculado

In [4]:
df["Calculated Total Amount"] = df["Price per Unit"] * df["Quantity"]

df.head(10)

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Calculated Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100
5,6,2023-04-25,CUST006,Female,45,Beauty,1,30,30,30
6,7,2023-03-13,CUST007,Male,46,Clothing,2,25,50,50
7,8,2023-02-22,CUST008,Male,30,Electronics,4,25,100,100
8,9,2023-12-13,CUST009,Male,63,Electronics,2,300,600,600
9,10,2023-10-07,CUST010,Female,52,Clothing,4,50,200,200


Se halla la diferencia entre el calor registrado y el calculado, luego se buscan: la media, el desvio estandar, el minimo y el maximo

In [5]:
diff = df["Total Amount"] - df["Calculated Total Amount"]
diff_absoluta = abs(diff)
diff_absoluta.agg(["mean", "std", "min", "max"])

mean    0.0
std     0.0
min     0.0
max     0.0
dtype: float64

Se observa que el valor calculado coincide plenamente con el valor registrado.

A continuacion realizaremos una clasificacion de las ventas.

In [6]:
sales_min = df["Total Amount"].min()
sales_max = df["Total Amount"].max()
print("Valor mínimo de ventas:", sales_min)
print("Valor máximo de ventas:", sales_max)
diff = sales_max - sales_min
print("Diferencia entre valor maximo y minimo:", diff)
limite_ventas_bajas = sales_min + diff / 3
limite_ventas_medias = limite_ventas_bajas + diff / 3
limite_ventas = sales_max
print("Intervalo Ventas bajas: hasta ", limite_ventas_bajas)
print("Intervalo Ventas medias: De ", limite_ventas_bajas, "hasta", limite_ventas_medias)
print("Intervalo Ventas altas: De ", limite_ventas_medias, "hasta ", limite_ventas)

Valor mínimo de ventas: 25
Valor máximo de ventas: 2000
Diferencia entre valor maximo y minimo: 1975
Intervalo Ventas bajas: hasta  683.3333333333334
Intervalo Ventas medias: De  683.3333333333334 hasta 1341.6666666666667
Intervalo Ventas altas: De  1341.6666666666667 hasta  2000


Estos grupos definen los niveles de ventas alto, medio y bajo. Estos grupos pasan a ser valores categoricos del dataframe, crearemos una columna para ello

In [7]:
df = df.assign(Sale_Category='Media')
df.loc[df['Total Amount'] < limite_ventas_bajas, 'Sale_Category'] = 'Baja'
df.loc[df['Total Amount'] > limite_ventas_medias, 'Sale_Category'] = 'Alta'
df.head(10)

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Calculated Total Amount,Sale_Category
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,Baja
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,Media
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,Baja
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,Baja
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,Baja
5,6,2023-04-25,CUST006,Female,45,Beauty,1,30,30,30,Baja
6,7,2023-03-13,CUST007,Male,46,Clothing,2,25,50,50,Baja
7,8,2023-02-22,CUST008,Male,30,Electronics,4,25,100,100,Baja
8,9,2023-12-13,CUST009,Male,63,Electronics,2,300,600,600,Baja
9,10,2023-10-07,CUST010,Female,52,Clothing,4,50,200,200,Baja


Se crea la columna categorica de Categoria de Venta, esto servira para una clasificacion rapida.

### Agrupación y Agregación

Se procede a agregar los grupos y a analizar los datos con agregaciones.

In [8]:
groupA =df.groupby("Product Category")["Total Amount"]
groupA.head(10)

0      150
1     1000
2       30
3      500
4      100
5       30
6       50
7      100
8      600
9      200
10     100
11      75
12    1500
13     120
14    2000
15    1500
16     100
17      50
18      50
19     900
20     500
24      50
25    1000
26      50
27     500
28      30
29     900
30    1200
31      90
32     100
Name: Total Amount, dtype: int64

Se realiza una agrupacion por categoria, a continuacion, se aplican funciones de media, desviacion, minimo y maximo.

In [9]:
groupA.agg(["mean", "std", "min", "max"])

,mean,std,min,max
Product Category,,,,
Beauty,467.475570,563.612788,25,2000
Clothing,443.247863,550.695917,25,2000
Electronics,458.786550,567.540150,25,2000


Se realizan otras agrupaciones con diversas columnas para observar y comprender los datos

In [10]:
df.groupby("Gender")["Age"].agg(["mean", "std", "min", "max"])

,mean,std,min,max
Gender,,,,
Female,41.356863,13.568067,18,64
Male,41.428571,13.812208,18,64


##### Agrupación y Agregación por Product Category

In [11]:
df.groupby("Sale_Category")["Total Amount"].agg(["mean", "std", "min", "max"])

,mean,std,min,max
Sale_Category,,,,
Alta,1747.474747,251.259454,1500,2000
Baja,154.076087,157.650788,25,600
Media,1027.878788,127.158380,900,1200


In [12]:
df.groupby("Sale_Category")["Age"].agg(["mean", "std", "min", "max"])

,mean,std,min,max
Sale_Category,,,,
Alta,39.202020,13.616451,18,64
Baja,41.841033,13.694251,18,64
Media,40.703030,13.589462,18,64


#### Análisis personalizado con apply.

Se crea una columna para realizar los analisis personalizados con apply.

In [13]:
def funcion_clasificacion_etaria(x):
    if x["Age"] < 24:
        return "Adulto Joven"
    elif 24 <= x["Age"] < 60:
        return "Adulto"
    elif 60 <= x["Age"]:
        return "Adulto mayor"

df["Age Classification"] = df.apply(funcion_clasificacion_etaria, axis=1)
df

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Calculated Total Amount,Sale_Category,Age Classification
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,Baja,Adulto
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,Media,Adulto
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,Baja,Adulto
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,Baja,Adulto
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,Baja,Adulto
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50,50,Baja,Adulto mayor
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90,90,Baja,Adulto
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100,100,Baja,Adulto Joven
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150,150,Baja,Adulto


Se halla la desviacion estandar de cada venta con respecto a su grupo

In [14]:
std_dev = df["Total Amount"].std()

print("Desviación estándar", std_dev)

df["Total Amount Stdev Difference"] = df.apply(lambda x: abs(x["Total Amount"] - std_dev), axis=1)
df

Desviación estándar 559.997631555123


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Calculated Total Amount,Sale_Category,Age Classification,Total Amount Stdev Difference
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,Baja,Adulto,409.997632
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,Media,Adulto,440.002368
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,Baja,Adulto,529.997632
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,Baja,Adulto,59.997632
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,Baja,Adulto,459.997632
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50,50,Baja,Adulto mayor,509.997632
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90,90,Baja,Adulto,469.997632
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100,100,Baja,Adulto Joven,459.997632
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150,150,Baja,Adulto,409.997632


In [15]:
#Guardamos el dataframe para usarlo en la siguiente parte
df.to_csv('data/retail_sales_dataset3.csv', sep=',', index=False, encoding='utf-8')